# Join Health Info with Census Tracts

In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster



In [2]:
# import shapefile, 500Cities_Tracts
health_info = pd.read_csv('../data/location/500_Cities__Local_Data_for_Better_Health__2019_release.csv')

health_info.head( )

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,High_Confidence_Limit,Data_Value_Footnote_Symbol,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
0,2017,GA,Georgia,Savannah,Census Tract,BRFSS,Prevention,1369000-13051004002,Visits to doctor for routine checkup within th...,%,...,75.5,NaN,NaN,"3,697","(32.0005963715, -81.1016216322)",PREVENT,CHECKUP,1369000.0,1.305100e+10,Annual Checkup
1,2017,HI,Hawaii,Honolulu,Census Tract,BRFSS,Health Outcomes,15003003101,Coronary heart disease among adults aged >=18 ...,%,...,5.6,NaN,NaN,"3,687","(21.3258064243, -157.793873751)",HLTHOUT,CHD,15003.0,1.500300e+10,Coronary Heart Disease
2,2016,GA,Georgia,Sandy Springs,City,BRFSS,Prevention,1368516,Papanicolaou smear use among adult women aged ...,%,...,85.1,NaN,NaN,"93,853","(33.9366669867, -84.3703197084)",PREVENT,PAPTEST,1368516.0,NaN,Pap Smear Test
3,2017,GA,Georgia,Roswell,Census Tract,BRFSS,Health Outcomes,1367284-13121011410,Mental health not good for >=14 days among adu...,%,...,9.8,NaN,NaN,"6,363","(33.9880169272, -84.2891245228)",HLTHOUT,MHLTH,1367284.0,1.312101e+10,Mental Health
4,2017,HI,Hawaii,Honolulu,Census Tract,BRFSS,Prevention,15003004800,Current lack of health insurance among adults ...,%,...,12.5,NaN,NaN,"6,707","(21.3415691226, -157.853573613)",PREVENT,ACCESS2,15003.0,1.500300e+10,Health Insurance


In [3]:
# Filter rows outside of Nashville
health_info = health_info[health_info.CityName == 'Nashville']
health_info

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,High_Confidence_Limit,Data_Value_Footnote_Symbol,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
10275,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037018700,Stroke among adults aged >=18 Years,%,...,2.6,NaN,NaN,565,"(36.0524304194, -86.8041894006)",HLTHOUT,STROKE,4752006.0,4.703702e+10,Stroke
15268,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037018102,Binge drinking among adults aged >=18 Years,%,...,13.6,NaN,NaN,"3,529","(36.1276431581, -86.8678623837)",UNHBEH,BINGE,4752006.0,4.703702e+10,Binge Drinking
20930,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019110,Chronic kidney disease among adults aged >=18 ...,%,...,3.1,NaN,NaN,"3,749","(36.0517956287, -86.7048611505)",HLTHOUT,KIDNEY,4752006.0,4.703702e+10,Chronic Kidney Disease
20994,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019003,Current asthma among adults aged >=18 Years,%,...,10.5,NaN,NaN,"5,032","(36.0769982456, -86.7054436498)",HLTHOUT,CASTHMA,4752006.0,4.703702e+10,Current Asthma
24780,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037014800,Obesity among adults aged >=18 Years,%,...,53.6,NaN,NaN,"3,127","(36.151096554, -86.7597078621)",UNHBEH,OBESITY,4752006.0,4.703701e+10,Obesity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675838,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037015609,Older adult women aged >=65 Years who are up t...,%,...,48.0,NaN,NaN,"3,411","(36.1719689182, -86.6206020339)",PREVENT,COREW,4752006.0,4.703702e+10,Core preventive services for older women
675839,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037018700,Mammography use among women aged 50–74 Years,%,...,83.1,NaN,NaN,565,"(36.0524304194, -86.8041894006)",PREVENT,MAMMOUSE,4752006.0,4.703702e+10,Mammography
675843,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037011800,Older adult men aged >=65 Years who are up to ...,%,...,24.2,NaN,NaN,"2,673","(36.1860223442, -86.7643746828)",PREVENT,COREM,4752006.0,4.703701e+10,Core preventive services for older men
675845,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037011400,Visits to doctor for routine checkup within th...,%,...,75.0,NaN,NaN,"4,461","(36.1985224304, -86.7337794886)",PREVENT,CHECKUP,4752006.0,4.703701e+10,Annual Checkup


In [4]:
# Filter city-level rows
health_info = health_info[health_info.GeographicLevel == 'Census Tract']
health_info

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,High_Confidence_Limit,Data_Value_Footnote_Symbol,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
10275,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037018700,Stroke among adults aged >=18 Years,%,...,2.6,NaN,NaN,565,"(36.0524304194, -86.8041894006)",HLTHOUT,STROKE,4752006.0,4.703702e+10,Stroke
15268,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037018102,Binge drinking among adults aged >=18 Years,%,...,13.6,NaN,NaN,"3,529","(36.1276431581, -86.8678623837)",UNHBEH,BINGE,4752006.0,4.703702e+10,Binge Drinking
20930,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019110,Chronic kidney disease among adults aged >=18 ...,%,...,3.1,NaN,NaN,"3,749","(36.0517956287, -86.7048611505)",HLTHOUT,KIDNEY,4752006.0,4.703702e+10,Chronic Kidney Disease
20994,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019003,Current asthma among adults aged >=18 Years,%,...,10.5,NaN,NaN,"5,032","(36.0769982456, -86.7054436498)",HLTHOUT,CASTHMA,4752006.0,4.703702e+10,Current Asthma
24780,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037014800,Obesity among adults aged >=18 Years,%,...,53.6,NaN,NaN,"3,127","(36.151096554, -86.7597078621)",UNHBEH,OBESITY,4752006.0,4.703701e+10,Obesity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675838,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037015609,Older adult women aged >=65 Years who are up t...,%,...,48.0,NaN,NaN,"3,411","(36.1719689182, -86.6206020339)",PREVENT,COREW,4752006.0,4.703702e+10,Core preventive services for older women
675839,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037018700,Mammography use among women aged 50–74 Years,%,...,83.1,NaN,NaN,565,"(36.0524304194, -86.8041894006)",PREVENT,MAMMOUSE,4752006.0,4.703702e+10,Mammography
675843,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037011800,Older adult men aged >=65 Years who are up to ...,%,...,24.2,NaN,NaN,"2,673","(36.1860223442, -86.7643746828)",PREVENT,COREM,4752006.0,4.703701e+10,Core preventive services for older men
675845,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037011400,Visits to doctor for routine checkup within th...,%,...,75.0,NaN,NaN,"4,461","(36.1985224304, -86.7337794886)",PREVENT,CHECKUP,4752006.0,4.703701e+10,Annual Checkup


In [5]:
# split GeoLocation column into Longitude and Latitude
# df[['First','Last']] = df.Name.str.split(expand=True) 
health_info[['Latitude','Longitude']] = health_info.GeoLocation.str.split(expand=True) 
health_info

/opt/anaconda3/envs/geospatial/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text,Latitude,Longitude
10275,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037018700,Stroke among adults aged >=18 Years,%,...,NaN,565,"(36.0524304194, -86.8041894006)",HLTHOUT,STROKE,4752006.0,4.703702e+10,Stroke,"(36.0524304194,",-86.8041894006)
15268,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037018102,Binge drinking among adults aged >=18 Years,%,...,NaN,"3,529","(36.1276431581, -86.8678623837)",UNHBEH,BINGE,4752006.0,4.703702e+10,Binge Drinking,"(36.1276431581,",-86.8678623837)
20930,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019110,Chronic kidney disease among adults aged >=18 ...,%,...,NaN,"3,749","(36.0517956287, -86.7048611505)",HLTHOUT,KIDNEY,4752006.0,4.703702e+10,Chronic Kidney Disease,"(36.0517956287,",-86.7048611505)
20994,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019003,Current asthma among adults aged >=18 Years,%,...,NaN,"5,032","(36.0769982456, -86.7054436498)",HLTHOUT,CASTHMA,4752006.0,4.703702e+10,Current Asthma,"(36.0769982456,",-86.7054436498)
24780,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037014800,Obesity among adults aged >=18 Years,%,...,NaN,"3,127","(36.151096554, -86.7597078621)",UNHBEH,OBESITY,4752006.0,4.703701e+10,Obesity,"(36.151096554,",-86.7597078621)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675838,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037015609,Older adult women aged >=65 Years who are up t...,%,...,NaN,"3,411","(36.1719689182, -86.6206020339)",PREVENT,COREW,4752006.0,4.703702e+10,Core preventive services for older women,"(36.1719689182,",-86.6206020339)
675839,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037018700,Mammography use among women aged 50–74 Years,%,...,NaN,565,"(36.0524304194, -86.8041894006)",PREVENT,MAMMOUSE,4752006.0,4.703702e+10,Mammography,"(36.0524304194,",-86.8041894006)
675843,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037011800,Older adult men aged >=65 Years who are up to ...,%,...,NaN,"2,673","(36.1860223442, -86.7643746828)",PREVENT,COREM,4752006.0,4.703701e+10,Core preventive services for older men,"(36.1860223442,",-86.7643746828)
675845,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037011400,Visits to doctor for routine checkup within th...,%,...,NaN,"4,461","(36.1985224304, -86.7337794886)",PREVENT,CHECKUP,4752006.0,4.703701e+10,Annual Checkup,"(36.1985224304,",-86.7337794886)


In [6]:
# .columns = health_info.columns.str.replace(' ', '_').str.lower()
health_info['Latitude'] = health_info['Latitude'].str.replace('(', '').str.replace(',', '')
health_info['Longitude'] = health_info['Longitude'].str.replace(')', '')
health_info

/opt/anaconda3/envs/geospatial/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/geospatial/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text,Latitude,Longitude
10275,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037018700,Stroke among adults aged >=18 Years,%,...,NaN,565,"(36.0524304194, -86.8041894006)",HLTHOUT,STROKE,4752006.0,4.703702e+10,Stroke,36.0524304194,-86.8041894006
15268,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037018102,Binge drinking among adults aged >=18 Years,%,...,NaN,"3,529","(36.1276431581, -86.8678623837)",UNHBEH,BINGE,4752006.0,4.703702e+10,Binge Drinking,36.1276431581,-86.8678623837
20930,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019110,Chronic kidney disease among adults aged >=18 ...,%,...,NaN,"3,749","(36.0517956287, -86.7048611505)",HLTHOUT,KIDNEY,4752006.0,4.703702e+10,Chronic Kidney Disease,36.0517956287,-86.7048611505
20994,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019003,Current asthma among adults aged >=18 Years,%,...,NaN,"5,032","(36.0769982456, -86.7054436498)",HLTHOUT,CASTHMA,4752006.0,4.703702e+10,Current Asthma,36.0769982456,-86.7054436498
24780,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037014800,Obesity among adults aged >=18 Years,%,...,NaN,"3,127","(36.151096554, -86.7597078621)",UNHBEH,OBESITY,4752006.0,4.703701e+10,Obesity,36.151096554,-86.7597078621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675838,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037015609,Older adult women aged >=65 Years who are up t...,%,...,NaN,"3,411","(36.1719689182, -86.6206020339)",PREVENT,COREW,4752006.0,4.703702e+10,Core preventive services for older women,36.1719689182,-86.6206020339
675839,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037018700,Mammography use among women aged 50–74 Years,%,...,NaN,565,"(36.0524304194, -86.8041894006)",PREVENT,MAMMOUSE,4752006.0,4.703702e+10,Mammography,36.0524304194,-86.8041894006
675843,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037011800,Older adult men aged >=65 Years who are up to ...,%,...,NaN,"2,673","(36.1860223442, -86.7643746828)",PREVENT,COREM,4752006.0,4.703701e+10,Core preventive services for older men,36.1860223442,-86.7643746828
675845,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037011400,Visits to doctor for routine checkup within th...,%,...,NaN,"4,461","(36.1985224304, -86.7337794886)",PREVENT,CHECKUP,4752006.0,4.703701e+10,Annual Checkup,36.1985224304,-86.7337794886


In [17]:
# exclude all but 2017
result = health_info[health_info.Year != '2016']
# = health_info['Latitude'].str.replace('(', '').str.replace(',', '')

result

/opt/anaconda3/envs/geospatial/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text,Latitude,Longitude,geometry
10275,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037018700,Stroke among adults aged >=18 Years,%,...,565,"(36.0524304194, -86.8041894006)",HLTHOUT,STROKE,4752006.0,4.703702e+10,Stroke,36.0524304194,-86.8041894006,POINT (-86.80419 36.05243)
15268,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037018102,Binge drinking among adults aged >=18 Years,%,...,"3,529","(36.1276431581, -86.8678623837)",UNHBEH,BINGE,4752006.0,4.703702e+10,Binge Drinking,36.1276431581,-86.8678623837,POINT (-86.86786 36.12764)
20930,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019110,Chronic kidney disease among adults aged >=18 ...,%,...,"3,749","(36.0517956287, -86.7048611505)",HLTHOUT,KIDNEY,4752006.0,4.703702e+10,Chronic Kidney Disease,36.0517956287,-86.7048611505,POINT (-86.70486 36.05180)
20994,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019003,Current asthma among adults aged >=18 Years,%,...,"5,032","(36.0769982456, -86.7054436498)",HLTHOUT,CASTHMA,4752006.0,4.703702e+10,Current Asthma,36.0769982456,-86.7054436498,POINT (-86.70544 36.07700)
24780,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037014800,Obesity among adults aged >=18 Years,%,...,"3,127","(36.151096554, -86.7597078621)",UNHBEH,OBESITY,4752006.0,4.703701e+10,Obesity,36.151096554,-86.7597078621,POINT (-86.75971 36.15110)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675838,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037015609,Older adult women aged >=65 Years who are up t...,%,...,"3,411","(36.1719689182, -86.6206020339)",PREVENT,COREW,4752006.0,4.703702e+10,Core preventive services for older women,36.1719689182,-86.6206020339,POINT (-86.62060 36.17197)
675839,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037018700,Mammography use among women aged 50–74 Years,%,...,565,"(36.0524304194, -86.8041894006)",PREVENT,MAMMOUSE,4752006.0,4.703702e+10,Mammography,36.0524304194,-86.8041894006,POINT (-86.80419 36.05243)
675843,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037011800,Older adult men aged >=65 Years who are up to ...,%,...,"2,673","(36.1860223442, -86.7643746828)",PREVENT,COREM,4752006.0,4.703701e+10,Core preventive services for older men,36.1860223442,-86.7643746828,POINT (-86.76437 36.18602)
675845,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037011400,Visits to doctor for routine checkup within th...,%,...,"4,461","(36.1985224304, -86.7337794886)",PREVENT,CHECKUP,4752006.0,4.703701e+10,Annual Checkup,36.1985224304,-86.7337794886,POINT (-86.73378 36.19852)


In [8]:
# save new dataframes
health_info.to_csv('../data/location/nashville_census_tracts.csv')

In [9]:
# import shapefile, 500Cities_Tracts
census_tracts = gpd.read_file('../data/location/500Cities_Tracts/500Cities_Tracts_Clip.shp')
print(census_tracts.crs)
census_tracts.head( )

epsg:3857


,place2010,tract2010,ST,PlaceName,plctract10,PlcTrPop10,geometry
0,0107000,01073000100,01,Birmingham,0107000-01073000100,3042,"POLYGON ((-9653001.056 3974630.899, -9652982.0..."
1,0107000,01073000300,01,Birmingham,0107000-01073000300,2735,"POLYGON ((-9657462.853 3968794.996, -9657470.2..."
2,0107000,01073000400,01,Birmingham,0107000-01073000400,3338,"POLYGON ((-9657275.613 3972217.160, -9657275.3..."
3,0107000,01073000500,01,Birmingham,0107000-01073000500,2864,"POLYGON ((-9660666.407 3968887.964, -9660631.7..."
4,0107000,01073000700,01,Birmingham,0107000-01073000700,2577,"POLYGON ((-9661391.097 3968591.429, -9661398.2..."


In [10]:
# will this make a difference?
census_tracts.crs = "EPSG:3857"
print(census_tracts.crs)


EPSG:3857


In [11]:
# Filter rows outside of Nashville
census_tracts = census_tracts[census_tracts.PlaceName == 'Nashville']
census_tracts

,place2010,tract2010,ST,PlaceName,plctract10,PlcTrPop10,geometry
23537,4752006,47037010103,47,Nashville,4752006-47037010103,2377,"POLYGON ((-9665101.952 4349117.051, -9665110.6..."
23538,4752006,47037010104,47,Nashville,4752006-47037010104,3070,"POLYGON ((-9663717.025 4341008.540, -9663723.7..."
23539,4752006,47037010105,47,Nashville,4752006-47037010105,4268,"POLYGON ((-9665970.800 4337243.674, -9665968.2..."
23540,4752006,47037010106,47,Nashville,4752006-47037010106,2668,"POLYGON ((-9660650.952 4338992.549, -9660629.5..."
23541,4752006,47037010201,47,Nashville,4752006-47037010201,4041,"POLYGON ((-9656019.726 4341048.317, -9655844.8..."
...,...,...,...,...,...,...,...
23691,4752006,47037019500,47,Nashville,4752006-47037019500,5916,"POLYGON ((-9658131.122 4322531.407, -9658130.5..."
23692,4752006,47037019600,47,Nashville,4752006-47037019600,3764,"POLYGON ((-9651287.086 4323212.955, -9651268.6..."
24585,4752006,47037010301,47,Nashville,4752006-47037010301,0,"MULTIPOLYGON (((-9655155.660 4340929.541, -965..."
24586,4752006,47037980100,47,Nashville,4752006-47037980100,0,"POLYGON ((-9647676.660 4320095.393, -9647190.8..."


In [12]:
# add geometry to health_info
health_info['geometry'] = health_info.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), axis=1)
health_info.tail(20)

/opt/anaconda3/envs/geospatial/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text,Latitude,Longitude,geometry
675779,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037013500,High cholesterol among adults aged >=18 Years ...,%,...,"1,936","(36.1503310917, -86.8286410961)",HLTHOUT,HIGHCHOL,4752006.0,4.703701e+10,High Cholesterol,36.1503310917,-86.8286410961,POINT (-86.8286410961 36.1503310917)
675781,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037010802,High blood pressure among adults aged >=18 Years,%,...,"3,274","(36.2509916244, -86.731871255)",HLTHOUT,BPHIGH,4752006.0,4.703701e+10,High Blood Pressure,36.2509916244,-86.731871255,POINT (-86.731871255 36.2509916244)
675784,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037016700,Current smoking among adults aged >=18 Years,%,...,"5,398","(36.1329172709, -86.8300431737)",UNHBEH,CSMOKING,4752006.0,4.703702e+10,Current Smoking,36.1329172709,-86.8300431737,POINT (-86.83004317370001 36.1329172709)
675788,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037018410,Obesity among adults aged >=18 Years,%,...,"2,770","(36.0712760339, -86.9382627391)",UNHBEH,OBESITY,4752006.0,4.703702e+10,Obesity,36.0712760339,-86.9382627391,POINT (-86.9382627391 36.0712760339)
675790,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037017500,Older adult men aged >=65 Years who are up to ...,%,...,"2,951","(36.1012139547, -86.7435789922)",PREVENT,COREM,4752006.0,4.703702e+10,Core preventive services for older men,36.1012139547,-86.7435789922,POINT (-86.7435789922 36.1012139547)
675791,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019003,Stroke among adults aged >=18 Years,%,...,"5,032","(36.0769982456, -86.7054436498)",HLTHOUT,STROKE,4752006.0,4.703702e+10,Stroke,36.0769982456,-86.7054436498,POINT (-86.7054436498 36.0769982456)
675795,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037010801,Taking medicine for high blood pressure contro...,%,...,"5,363","(36.2758406196, -86.7227516725)",PREVENT,BPMED,4752006.0,4.703701e+10,Taking BP Medication,36.2758406196,-86.7227516725,POINT (-86.7227516725 36.2758406196)
675799,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037015617,Visits to dentist or dental clinic among adult...,%,...,"2,368","(36.1017022136, -86.6046378648)",PREVENT,DENTAL,4752006.0,4.703702e+10,Dental Visit,36.1017022136,-86.6046378648,POINT (-86.6046378648 36.1017022136)
675805,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037012200,Taking medicine for high blood pressure contro...,%,...,"2,056","(36.1701688187, -86.7363924126)",PREVENT,BPMED,4752006.0,4.703701e+10,Taking BP Medication,36.1701688187,-86.7363924126,POINT (-86.7363924126 36.1701688187)
675808,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037011100,Taking medicine for high blood pressure contro...,%,...,"3,921","(36.2246054475, -86.7196053106)",PREVENT,BPMED,4752006.0,4.703701e+10,Taking BP Medication,36.2246054475,-86.7196053106,POINT (-86.7196053106 36.2246054475)


In [13]:
# create geodataframe from health_info
health_info_geo = gpd.GeoDataFrame(health_info, 
                           crs = census_tracts.crs, 
                           geometry = health_info['geometry'])

In [14]:
health_info_geo

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text,Latitude,Longitude,geometry
10275,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037018700,Stroke among adults aged >=18 Years,%,...,565,"(36.0524304194, -86.8041894006)",HLTHOUT,STROKE,4752006.0,4.703702e+10,Stroke,36.0524304194,-86.8041894006,POINT (-86.80419 36.05243)
15268,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037018102,Binge drinking among adults aged >=18 Years,%,...,"3,529","(36.1276431581, -86.8678623837)",UNHBEH,BINGE,4752006.0,4.703702e+10,Binge Drinking,36.1276431581,-86.8678623837,POINT (-86.86786 36.12764)
20930,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019110,Chronic kidney disease among adults aged >=18 ...,%,...,"3,749","(36.0517956287, -86.7048611505)",HLTHOUT,KIDNEY,4752006.0,4.703702e+10,Chronic Kidney Disease,36.0517956287,-86.7048611505,POINT (-86.70486 36.05180)
20994,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019003,Current asthma among adults aged >=18 Years,%,...,"5,032","(36.0769982456, -86.7054436498)",HLTHOUT,CASTHMA,4752006.0,4.703702e+10,Current Asthma,36.0769982456,-86.7054436498,POINT (-86.70544 36.07700)
24780,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037014800,Obesity among adults aged >=18 Years,%,...,"3,127","(36.151096554, -86.7597078621)",UNHBEH,OBESITY,4752006.0,4.703701e+10,Obesity,36.151096554,-86.7597078621,POINT (-86.75971 36.15110)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675838,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037015609,Older adult women aged >=65 Years who are up t...,%,...,"3,411","(36.1719689182, -86.6206020339)",PREVENT,COREW,4752006.0,4.703702e+10,Core preventive services for older women,36.1719689182,-86.6206020339,POINT (-86.62060 36.17197)
675839,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037018700,Mammography use among women aged 50–74 Years,%,...,565,"(36.0524304194, -86.8041894006)",PREVENT,MAMMOUSE,4752006.0,4.703702e+10,Mammography,36.0524304194,-86.8041894006,POINT (-86.80419 36.05243)
675843,2016,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037011800,Older adult men aged >=65 Years who are up to ...,%,...,"2,673","(36.1860223442, -86.7643746828)",PREVENT,COREM,4752006.0,4.703701e+10,Core preventive services for older men,36.1860223442,-86.7643746828,POINT (-86.76437 36.18602)
675845,2017,TN,Tennessee,Nashville,Census Tract,BRFSS,Prevention,4752006-47037011400,Visits to doctor for routine checkup within th...,%,...,"4,461","(36.1985224304, -86.7337794886)",PREVENT,CHECKUP,4752006.0,4.703701e+10,Annual Checkup,36.1985224304,-86.7337794886,POINT (-86.73378 36.19852)


In [15]:
result = pd.concat([health_info_geo, census_tracts])

result

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Short_Question_Text,Latitude,Longitude,geometry,place2010,tract2010,ST,PlaceName,plctract10,PlcTrPop10
10275,2017.0,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037018700,Stroke among adults aged >=18 Years,%,...,Stroke,36.0524304194,-86.8041894006,POINT (-86.804 36.052),NaN,NaN,NaN,NaN,NaN,NaN
15268,2017.0,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037018102,Binge drinking among adults aged >=18 Years,%,...,Binge Drinking,36.1276431581,-86.8678623837,POINT (-86.868 36.128),NaN,NaN,NaN,NaN,NaN,NaN
20930,2017.0,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019110,Chronic kidney disease among adults aged >=18 ...,%,...,Chronic Kidney Disease,36.0517956287,-86.7048611505,POINT (-86.705 36.052),NaN,NaN,NaN,NaN,NaN,NaN
20994,2017.0,TN,Tennessee,Nashville,Census Tract,BRFSS,Health Outcomes,4752006-47037019003,Current asthma among adults aged >=18 Years,%,...,Current Asthma,36.0769982456,-86.7054436498,POINT (-86.705 36.077),NaN,NaN,NaN,NaN,NaN,NaN
24780,2017.0,TN,Tennessee,Nashville,Census Tract,BRFSS,Unhealthy Behaviors,4752006-47037014800,Obesity among adults aged >=18 Years,%,...,Obesity,36.151096554,-86.7597078621,POINT (-86.760 36.151),NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"POLYGON ((-9658131.122 4322531.407, -9658130.5...",4752006,47037019500,47,Nashville,4752006-47037019500,5916.0
23692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"POLYGON ((-9651287.086 4323212.955, -9651268.6...",4752006,47037019600,47,Nashville,4752006-47037019600,3764.0
24585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"MULTIPOLYGON (((-9655155.660 4340929.541, -965...",4752006,47037010301,47,Nashville,4752006-47037010301,0.0
24586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"POLYGON ((-9647676.660 4320095.393, -9647190.8...",4752006,47037980100,47,Nashville,4752006-47037980100,0.0


In [16]:
# save new dataframes
# result.to_file("../data/location/all_providers_tracts.geojson", driver='GeoJSON')
result.to_file("../data/location/nashville_census_tracts.shp")


RuntimeError: Failed to write record: {'id': '23537', 'type': 'Feature', 'properties': {'Category': None, 'CategoryID': None, 'CityFIPS': None, 'CityName': None, 'DataSource': None, 'DataValueTypeID': None, 'Data_Value': None, 'Data_Value_Footnote': None, 'Data_Value_Footnote_Symbol': None, 'Data_Value_Type': None, 'Data_Value_Unit': None, 'GeoLocation': None, 'GeographicLevel': None, 'High_Confidence_Limit': None, 'Latitude': None, 'Longitude': None, 'Low_Confidence_Limit': None, 'Measure': None, 'MeasureId': None, 'PlaceName': 'Nashville', 'PlcTrPop10': 2377.0, 'PopulationCount': None, 'ST': '47', 'Short_Question_Text': None, 'StateAbbr': None, 'StateDesc': None, 'TractFIPS': None, 'UniqueID': None, 'Year': None, 'place2010': '4752006', 'plctract10': '4752006-47037010103', 'tract2010': '47037010103'}, 'geometry': {'type': 'Polygon', 'coordinates': (((-9665101.9518, 4349117.050999999), (-9665110.6347, 4349101.432499997), (-9665119.429, 4349085.952200003), (-9665133.4552, 4349065.08129